In [126]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from tqdm import tqdm

In [129]:
inception_v3_df = pd.read_csv("data/pred_inception_v3.csv")
inception_resnet_v2_df = pd.read_csv("data/pred_inception_resnet_v2.csv")
resnet50_df = pd.read_csv("data/pred_resnet50.csv")
xception_df = pd.read_csv("data/pred_xception.csv")
mix_df = pd.read_csv("data/mix_pred.csv")

In [43]:
print(np.array(inception_v3_df.query("img=='img_10004.jpg'").iloc[0, 1:]))
print(np.array(inception_resnet_v2_df.query("img=='img_10004.jpg'").iloc[0, 1:]))

[0.004999999888241291 0.004999999888241291 0.99500000476837158
 0.004999999888241291 0.004999999888241291 0.004999999888241291
 0.004999999888241291 0.004999999888241291 0.004999999888241291
 0.004999999888241291]
[0.19713923335075376 0.011285470798611641 0.64479970932006836
 0.0093476539477705956 0.020656859502196312 0.014582724310457706
 0.021793121472001083 0.010969457216560841 0.046836279332637787
 0.022589495405554768]


In [78]:
values = np.mean(np.array([inception_v3_df.query("img=='img_10004.jpg'").iloc[0, 1:], 
                  inception_resnet_v2_df.query("img=='img_10004.jpg'").iloc[0, 1:]])
        , axis=0)
print(values)
m = np.amax(values)
print(m)
for i, v in enumerate(values):
    if v < m:
        values[i] = 0.005
    else:
        values[i] = 0.995
        
values

[0.10106961661949752 0.008142735343426466 0.81989985704421997
 0.0071738269180059433 0.012828429695218801 0.0097913620993494987
 0.013396560680121187 0.0079847285524010658 0.025918139610439539
 0.01379474764689803]
0.819899857044


array([0.005, 0.005, 0.995, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005], dtype=object)

In [132]:
merged_pd = DataFrame(columns=inception_v3_df.columns)

for i in tqdm(inception_v3_df.index.tolist()):
    img_name = inception_v3_df.iloc[i, 0]
    values = np.mean(np.array([
        inception_v3_df.iloc[i, 1:], 
#         inception_resnet_v2_df.iloc[i, 1:],
        xception_df.iloc[i, 1:],
        mix_df.iloc[i, 1:]
#         resnet50_df.iloc[i, 1:]
    ]), axis=0)
    m = np.amax(values)
    for i, v in enumerate(values):
        if v < m:
            values[i] = 0.005
        else:
            values[i] = 0.995
    
    data = Series(np.concatenate([np.array([img_name]), values]), index=inception_v3_df.columns)
    merged_pd = merged_pd.append(data, ignore_index=True)
    
merged_pd.to_csv('data/pred_merged.csv', index=None)

100%|███████████████████████████████████| 79726/79726 [07:02<00:00, 188.77it/s]
